###Part 1
(1) print statements
(2) Pdb and breakpoint() as an alternative to print statements
Debug in Pytorch:
##Data:
(1) Device
(2) Data type
(3) Data shape
##Model:
(1) Device
(2) mode
##Loss:
(1) Activation
(2) Data type 
##Specific to HW1:
(1) Boardcasting, matrix multiplication
(2) Shape
(3) Row & Column Vector



# Part 2

- [x] Iterators and next to check if dataloaders is okay
- [x] Cover wandb
- [x] Cover way to track different model parameters using google sheets: [Google sheet](https://docs.google.com/spreadsheets/d/1V5KQAmiOKVtq-cVcn9IvmbeEUzjb_u_ofck0hDgwgOI/edit?usp=sharing)

# 0 Import libraries and setup

In [5]:
import sys
import time
import random

import numpy as np

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

# Make sure you choose suitable num_worker, otherwise it will result in errors
num_workers = 8 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  8  system version =  3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]


## 0.1 Wandb installation and login

1. Install wandb
1. Ensure you have created an account at [wandb.ai](https://wandb.ai/site) and you're logged in
1. Paste your API key in the text area

In [2]:
# !pip install wandb --upgrade
!pip install --upgrade wandb==0.10.8

import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 75 kB 1.9 MB/s 
     |████████████████████████████████| 133 kB 34.5 MB/s 
     |████████████████████████████████| 97 kB 3.6 MB/s 
     |████████████████████████████████| 170 kB 47.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=3fd9f5360d230ae4395afd604ae112f69b7d56811c7fbb05172a28c37badc17d
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
Successfully built subprocess32


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 0.2 Wandb setup 

In [3]:
# Initialize WandB 
config = {
    'batch_size': 5,
    'epochs': 15,
    'learning_rate': 0.001,
    'trial': 6 ,
    'train_size': 250,
    }

wandb.init(name='#6', 
          project="debugging-recitation", 
          notes='Simple MLP', 
          tags=['getting-started', 'MLP'],
          config=config)
config = wandb.config

wandb: Currently logged in as: ang9867 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# 1 Load data

In [7]:
train_size = config.train_size
dev_size = train_size // 3

train_data = np.random.uniform(1, 10, size=(train_size, 3))
train_labels = np.random.randint(2, size=train_size)


dev_data = np.random.uniform(1, 10, size=(dev_size, 3))
dev_labels =  np.random.randint(2, size=dev_size)

print('*'*50)
print('Training data')
print(train_data.shape, train_labels.shape)

print('*'*50)
print('Validation data')
print(dev_data.shape, dev_labels.shape)
print('='*50)

**************************************************
Training data
(250, 3) (250,)
**************************************************
Validation data
(83, 3) (83,)


# 2 Custom dataset class

In [8]:
class MyDataset(Dataset):
  def __init__(self, X, Y):
    self.X = torch.FloatTensor(X)
    self.Y = torch.LongTensor(Y)

  def __len__(self):
    return len(self.Y)

  def __getitem__(self, index):
    x = self.X[index]
    y = self.Y[index]

    return x, y

# 3 Dataloaders

In [9]:
batch_size = config.batch_size

train_dataset = MyDataset(train_data, train_labels)
train_args = dict(shuffle=True, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True) if cuda\
                else dict(shuffle=True, batch_size=batch_size, drop_last=True)
train_loader = DataLoader(train_dataset, **train_args)

dev_dataset = MyDataset(dev_data, dev_labels)
dev_args = dict(shuffle=False, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True) if cuda\
                else dict(shuffle=False, batch_size=batch_size, drop_last=True)
dev_loader = DataLoader(dev_dataset, **dev_args)

## 3.1 Debug iterator

In [10]:
train_iterator = iter(train_loader)
x, y = next(train_iterator)

print('***Assert that returned data is a tensor***')
assert isinstance(x, torch.Tensor)

print('***Assert that returned data shape is correct***')
assert x.shape[0] == batch_size, f"The batch size of X should be {batch_size} while gets {x.shape[0]}"
assert x.shape[1] == train_data.shape[1], f"The data length of X should be {train_data.shape[1]} while gets {x.shape[1]}"
assert y.shape[0] == batch_size, f"The batch size of Y should be {batch_size} while gets {y.shape[0]}"

print('***Assert that returned data type is correct***')
assert x.dtype == torch.float32 and y.dtype == torch.long, "x should be float32 tensor and y should be long tensor"

print(x.shape, y.shape)
print(x.dtype, y.dtype)
print(x.device, y.device)

***Assert that returned data is a tensor***
***Assert that returned data shape is correct***
***Assert that returned data type is correct***
torch.Size([5, 3]) torch.Size([5])
torch.float32 torch.int64
cpu cpu


In [11]:
dev_iterator = iter(dev_loader)
x, y = next(dev_iterator)

print('***Assert that returned data is a tensor***')
assert isinstance(x, torch.Tensor)

print('***Assert that returned data shape is correct***')
assert x.shape[0] == batch_size, f"The batch size of X should be {batch_size} while gets {x.shape[0]}"
assert x.shape[1] == train_data.shape[1], f"The data length of X should be {train_data.shape[1]} while gets {x.shape[1]}"
assert y.shape[0] == batch_size, f"The batch size of Y should be {batch_size} while gets {y.shape[0]}"

print('***Assert that returned data type is correct***')
assert x.dtype == torch.float32 and y.dtype == torch.long, "x should be float32 tensor and y should be long tensor"

print(x.shape, y.shape)

***Assert that returned data is a tensor***
***Assert that returned data shape is correct***
***Assert that returned data type is correct***
torch.Size([5, 3]) torch.Size([5])


# 4 Model definition

In [12]:
class MLP(nn.Module):
  def __init__(self, size):
    super(MLP, self).__init__()

    self.model = nn.Sequential(
        nn.Linear(size[0], size[1]),
        nn.ReLU(),
        nn.Linear(size[1], size[2])
    )

  def forward(self, x):
    return self.model(x)


## 4.1 Debug model

In [13]:
size = [3, 128, 2]

model = MLP(size)

model = model.to(device)

x = x.to(device)
#device = torch.device("cuda")

# make sure the model and data are on the same device
assert next(model.parameters()).device == x.device, "model and data should be on the same device"

output = model(x)

# check if returned values are 2 as expected
assert(output.shape[1] == 2)

print(output) 

tensor([[-1.6597, -2.4219],
        [-3.4394, -5.3311],
        [-1.9698, -3.4556],
        [-1.0606, -1.7359],
        [-3.0236, -4.3424]], device='cuda:0', grad_fn=<AddmmBackward>)


## 4.2 Model hyperparameters

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# Log the network weight histograms
wandb.watch(model)

print(model)

MLP(
  (model): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)


# 5 Model training

## 5.1 Train the model

In [15]:
def train_model(model, loader, criterion, optimizer):
  model.train()

  avg_loss = 0.0

  start = time.time()

  for batch_index, (inputs, targets) in enumerate(loader):
    optimizer.zero_grad()

    # put everything in one device
    inputs = inputs.to(device)
    targets = targets.to(device)

    outputs = model(inputs)

    # calculate loss
    loss = criterion(outputs, targets)
    avg_loss += loss.item()

    loss.backward()
    optimizer.step()

  end = time.time()
  avg_loss = np.mean(avg_loss)

  print('Training loss: {:.2f}, Time: {}'.format(avg_loss, end-start))
  return avg_loss

## 5.2 Validate the model

In [16]:
def validate_model(model, loader, criterion):
  with torch.no_grad():
    model.eval()

    avg_loss = 0.0
    actuals = []
    predictions = []

    start = time.time()

    for batch_index, (inputs, targets) in enumerate(loader):
      # put everything in one device
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)

      # calculate loss
      loss = criterion(outputs, targets)
      avg_loss += loss.item()

      outputs = outputs.cpu().numpy()
      targets = targets.cpu().numpy()
  
      outputs = np.argmax(outputs, axis=1)

      # reshape for stacking
      targets = targets.reshape((len(targets), 1))
      outputs = outputs.reshape((len(outputs), 1))

      actuals.append(targets)
      predictions.append(outputs)

    end = time.time()
    avg_loss = np.mean(avg_loss)

    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    acc = accuracy_score(actuals, predictions) * 100

    print('Validation loss: {:.2f}, Validation accuracy: {:.2f}%, Time: {}'.format(avg_loss, acc, end-start))
    return avg_loss, acc

## 5.3 Train and validate

In [17]:
epochs = config.epochs

max_accuracy = -1

for epoch in range(epochs):
  print('Epoch #{}'.format(epoch+1))

  # training_loss = train_model(train_loader, model)
  training_loss = train_model(model, train_loader, criterion, optimizer)
  val_loss, val_acc = validate_model(model, dev_loader, criterion)

  if val_acc > max_accuracy:
    max_accuracy = val_acc
    wandb.run.summary["best_accuracy"] = val_acc
    wandb.run.summary["best_epoch"] = epoch + 1


  # Log the loss and accuracy values at the end of each epoch
  wandb.log({"Epoch": epoch + 1, 
             "Train Loss": training_loss, 
             "Validation Loss: ": val_loss, 
             "Validation Accuracy": val_acc})   

  print('='*80)

print('Done')

Epoch #1
Training loss: 38.13, Time: 0.6622185707092285
Validation loss: 11.69, Validation accuracy: 42.50%, Time: 0.3575708866119385
Epoch #2
Training loss: 35.87, Time: 0.6745498180389404
Validation loss: 11.50, Validation accuracy: 47.50%, Time: 0.3699202537536621
Epoch #3
Training loss: 36.00, Time: 0.5732958316802979
Validation loss: 11.10, Validation accuracy: 53.75%, Time: 0.33397626876831055
Epoch #4
Training loss: 35.09, Time: 0.5990347862243652
Validation loss: 11.05, Validation accuracy: 58.75%, Time: 0.3844015598297119
Epoch #5
Training loss: 35.96, Time: 0.5543324947357178
Validation loss: 11.61, Validation accuracy: 47.50%, Time: 0.3481466770172119
Epoch #6
Training loss: 35.56, Time: 0.5591955184936523
Validation loss: 12.75, Validation accuracy: 40.00%, Time: 0.36812353134155273
Epoch #7
Training loss: 36.86, Time: 0.5582787990570068
Validation loss: 10.91, Validation accuracy: 57.50%, Time: 0.36353039741516113
Epoch #8
Training loss: 35.91, Time: 0.5419702529907227
Val